# Using Django commands for healthchecks


- Business logic checks for your code

For example, envision a scenario where you want to monitor the number of new orders within a specific timeframe:

In [ ]:
new_orders_count = Order.objects.filter(updated_at__gte=datetime.now(), shipped=True).count()

Implementing this healthcheck as a Django command has multiple benefits. Firstly, it offers a structured and organized approach to encapsulate business logic, ensuring maintainability and readability. Secondly, commands facilitate consistent execution, making it easy to reuse and integrate healthchecks across different parts of your application.

# Using commands to run healthchecks


In the world of Django, flexibility is yours, especially when it comes to running healthchecks. Django commands are an excellent tool for performing these checks conveniently and efficiently, while still giving developers total control over the process.

You can create your own commands using the ```BaseCommand``` class.

```BaseCommand``` is a base class in Django for creating your own custom commands using Django management commands. The class provides structures and methods you need to define your command's behavior.

Here are the main components of BaseCommand:

- The ```help``` attribute provides a brief description of your command's function. This description appears when you run the command with the --help option.

- The ```handle``` method is triggered when your command is executed. You can modify this method to determine your command's behavior. This method accepts arguments and parameters passed from the command line and usually uses them to set up and execute the task.

- The ```self.stdout``` and ```self.style``` attributes provide access to command output and formatting styles. You can use them to print messages, warnings, and errors to the console.

To create a command, create a new module inside your Django application, for example ```<yourapp>/management/commands/healthcheck.py.``` In this module, define a new class that derives from ```BaseCommand```. This class will represent your healthcheck command.

If you want to monitor the number of new orders in your custom management command, you can include a query on the Order model, like this:

In [ ]:
from yourapp.models import Order
from django.core.management.base import BaseCommand
from datetime import datetime

class Command(BaseCommand):
    help = 'Perform health checks for new orders'

    def handle(self, *args, **options):
        new_orders_count = Order.objects.filter(updated_at__gte=datetime.now(), shipped=True).count()
        self.stdout.write(self.style.SUCCESS(f'Number of new orders: {new_orders_count}'))

Now you can execute your healthcheck command using manage.py:


```bash
python manage.py healthcheck



!!! ```You can schedule this command to run routinely and gather the results to ensure that your system is operating within expected parameters and swiftly respond to any issues in your application's business logic. It's crucial to underline that optimal use of these checks is not for manual execution by individuals, but as a fundamental component of ongoing operations aimed at monitoring the overall health of the application. As such, the format might not be restricted to printed messages; instead, it could involve structures like dictionaries conveying specific states or the submission of metrics to an external system for further analysis.```

Creating your own commands using BaseCommand allows you to control various aspects of command execution in a Django application, making them a powerful tool to automate tasks and control your application from the command line.

Now, you know how to run a simple command, but can you modify it? Yes, you can.

The most straightforward method is to add arguments to our command. This can be useful, for example, to pass parameters to a healthcheck to change its behavior.

You can utilize the add_arguments method to define additional arguments that your command can accept:

In [ ]:
from django.core.management.base import BaseCommand

class Command(BaseCommand):
    help = 'Run healthchecks for business logic'

    def add_arguments(self, parser):
        parser.add_argument('my_arg', type=int, help='Description of the argument')

    def handle(self, *args, **options):
        my_arg_value = options['my_arg']
        self.stdout.write(self.style.SUCCESS(f'The provided argument value is: {my_arg_value}'))


To run this command, use:
```bash
python manage.py healthcheck 4

There are more argument types besides integers, but mostly used are strings and integers.

You can combine multiple healthchecks into a single command for more thorough testing:

In [ ]:
class Command(BaseCommand):
    help = 'Run multiple healthchecks'

    def handle(self, *args, **options):
        result1 = perform_healthcheck1()
        result2 = perform_healthcheck2()

        if result1 and result2:
            self.stdout.write(self.style.SUCCESS('All healthchecks passed'))
        else:
            self.stdout.write(self.style.ERROR('Some healthchecks failed'))

Here, ```perform_healthcheck1``` and ```perform_healthcheck2``` represent your business logic functions to execute various checks.

Often, you will need to run your command with detailed logs. To do this, you need to add a ```--verbose ```argument and use Django's built-in ```logging``` functions:

In [ ]:
class Command(BaseCommand):
    help = 'Description of your command'

    def add_arguments(self, parser):
        parser.add_argument('--verbose', action='store_true', help='Enable verbose mode')

    def handle(self, *args, **options):
        # creating a logger Instance
        logger = logging.getLogger(__name__)

        # checking if verbose mode is activated
        if options['verbose']:
            logger.setLevel(logging.DEBUG)

        # using a logger
        logger.info('This is an info message')
        logger.warning('This is a warning message')
        logger.error('This is an error message')

In this example, we create a logger instance using logging.getLogger(__name__). We then use the info, warning, and error methods to log messages of different levels. These messages will automatically print to the console when your command is run with the appropriate logging level.

Now, you can run your command with the ```--verbose``` argument:
```bash
python manage.py healthcheck --verbose

Verbose logging becomes particularly useful for healthchecks when you need a more detailed view of the execution process. In verbose mode, the command can provide additional logging, status updates, or debug information, making it easier to spot and diagnose problems. This is especially helpful when healthchecks are part of an automated monitoring system, as verbose output can give insights into the behavior of an application, aiding in pinpointing potential issues more effectively. The --verbose argument gives users the option to see this detailed information during the execution of the healthcheck.

When using the --verbose argument, note that it follows the store_true behavior, meaning that it acts as a flag. Flags are typically used for boolean options where the presence of the flag indicates a True value. In this case, the --verbose flag indicates whether verbose output should be enabled during the execution of the healthcheck.

# Command examples
### Command to send mail


Sending email notifications is a critical part of user interaction. To ensure this essential functionality operates smoothly, Django offers handy tools in the form of commands that can be used for healthchecks.

Such command can be useful for:

- ```Integration into monitoring processes``` - you can automate running such a command and incorporate it into your monitoring processes. Running this comprehensive command can alert you to problems with sending notifications before users start experiencing issues.

- ```Testing during development``` - commands can be useful during development, allowing you to quickly and easily verify that your application is set up correctly to send email.

- ```Tracking changes in mail settings``` - when you make alterations to the mail server configuration, these healthchecks can help you keep track of how these changes impact the functionality of sending notifications.

In [ ]:
from django.core.management.base import BaseCommand
from django.core.mail import send_mail

class Command(BaseCommand):
    help = 'Run healthcheck for email sending'

    def handle(self, *args, **options):
        try:
            # simple email sending verification
            send_mail(
                'Subject',
                'Message',
                'from@example.com',
                ['to@example.com'],
                fail_silently=False,
            )
            self.stdout.write(self.style.SUCCESS('Email healthcheck successfully passed'))
        except Exception as e:
            self.stderr.write(f'Email healthcheck failed with an exception: {e}')

This example uses the send_mail function to try to send a test email. If the email is sent successfully, the command will report success of the healthcheck. If not, it will display an error message.

### Command to dump user statistics

One of the critical aspects of ensuring the effectiveness of a web application is regular monitoring and analyzing of statistical data. To do this, we can use Django commands to download and assess user behaviour statistics, which allow for early detection of potential problems and ensure smooth operation of the application.

Regular testing of user statistics is important for the following reasons:

- Regular checks of user statistics uploads can expose hidden anomalies or data problems that could arise due to changes in the application's logic or unusual situations.

- The analysis of statistical data is an essential tool for assessing application performance. Healthchecks focused on statistics downloads help monitor execution time and data volume, therefor enabling a prompt response to performance changes.

- Regular checks of user statistics allow you to quickly identify factors impacting user experience. These could be related to page performance, response time, or interactions with key features of the app.

Let's assume you have a function ```export_user_statistics()``` that exports user statistics. The command to dump current user statistics would look like this:

In [ ]:
from django.core.management.base import BaseCommand
from yourapp.statistics import export_user_statistics

class Command(BaseCommand):
    help = 'Run healthcheck for user statistics export'

    def handle(self, *args, **options):
        try:
            export_user_statistics()
            self.stdout.write(self.style.SUCCESS('User statistics healthcheck successfully passed'))
        except Exception as e:
            self.stderr.write(f'User statistics healthcheck failed with an exception: {e}')

# Command to update engagement counts
Finally, user engagement is becoming one of the key success factors. Hence, to ensure the accuracy and freshness of engagement counts, Django offers tools in the form of healthcheck commands that update these numbers, providing valuable insights about how compelling the app is for its users.

Let's say you have a function update_engagement_counters() that updates user engagement counters. The counters updated by this command could include several metrics, such as number of visits, interactions with content, etc.

In [ ]:
from django.core.management.base import BaseCommand
from yourapp.engagement import update_engagement_counters

class Command(BaseCommand):
    help = 'Run healthcheck for updating engagement counters'

    def handle(self, *args, **options):
        try:
            update_engagement_counters()
            self.stdout.write(self.style.SUCCESS('Engagement counters healthcheck successfully passed'))
        except Exception as e:
            self.stderr.write(f'Engagement counters healthcheck failed with an exception: {e}')

This sample command to update engagement counters demonstrates how healthchecks can interact directly with your application's business logic to ensure the app's capacity to effectively interact with users and meet their needs.